### Pokemon API

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


Met behulp van de Pokemon API kunnen stats over Pokemon worden verkregen. Hiervoor is een request nodig. Met behulp van een request wordt de benodigde data opgevraagd. In de codecell hieronder vindt u een vraag om de hoeveelheid Pokemon. De hoeveelheid Pokemon in de database wordt teruggegeven aan de hand van de sleutel "count". Dit is een van de types die je op kunt halen in de APIResourceList (PokeAPI. (n.d.).).
Er wordt overigens een json-file teruggegeven. Dit is een veelgebruikt format om gestructureerde tekstuele data van internetbronnen te verkrijgen.


In [ ]:
import requests

def get_total_pokemon_count():
    base_url = "https://pokeapi.co/api/v2/pokemon/"
    response = requests.get(base_url)
    data = response.json()
    return data["count"]

total_pokemon = get_total_pokemon_count()
print(f"Totaal aantal Pokémon: {total_pokemon}")


De onderstaande code geeft over de Pokemon uit de API-app eigenschappen per Pokemon terug. Het id-nummer, de naam, het gewicht, het type pokemon, de stats en de mogelijke moves van die Pokemon owrden teruggegeven. De stats voor de Pokemon worden een voor een opgehaald. Als er geen volgende pokemon is, wat wordt gecheckt door middel van het APIresource type command "next" ( (PokeAPI. (n.d.).).), wordt er een DataFrame teruggegeven dat de gewilde statistieken bevat. Wij hebben voor deze statistieken in dit hoofdstuk gekozen, omdat de andere hoofdstukken al andere stats opvragen. Door te werken met een DataFrame kunnen wij de andere statistieken die op de andere manieren zijn verkregen relatief eenvoudig toevoegen aan een overkoepelend DataFrame. 

In [ ]:
import requests
import pandas as pd

def get_all_pokemon_data():
    base_url = "https://pokeapi.co/api/v2/pokemon/"
    all_data = []
    offset = 0
    limit = 100

    while True:
        response = requests.get(f"{base_url}?limit={limit}&offset={offset}")
        response_data = response.json()
        pokemon_list = response_data["results"]

        for pokemon in pokemon_list:
            try:
                pokemon_data = requests.get(pokemon["url"]).json()
                pokemon_entry = {
                    "id": pokemon_data["id"],
                    "name": pokemon_data["name"],
                    "weight": pokemon_data["weight"],
                    "types": [t["type"]["name"] for t in pokemon_data["types"]],
                    "stats": {stat["stat"]["name"]: stat["base_stat"] for stat in pokemon_data["stats"]},
                    "moves": [move["move"]["name"] for move in pokemon_data["moves"]],
                }
                all_data.append(pokemon_entry)
            except Exception as e:
                print(f"Fout bij ophalen van {pokemon['name']}: {e}")

        if response_data["next"] is None:
            break
        else:
            offset += limit

    return pd.DataFrame(all_data)


df_all_pokemon = get_all_pokemon_data()
print(df_all_pokemon.head())  


In [ ]:
df_all_pokemon

In [ ]:
def split_types(df):
    """
    Splits the 'types' column into two new columns: 'type1' and 'type2'.
    
    If there is only one type, 'type2' will be assigned NaN.
    
    Args:
    - df (pd.DataFrame): The DataFrame containing the 'types' column as a list.
    
    Returns:
    - pd.DataFrame: The original DataFrame with two new columns: 'type1' and 'type2'.
    """
    df['type1'] = df['types'].apply(lambda x: x[0] if len(x) > 0 else np.nan)
    df['type2'] = df['types'].apply(lambda x: x[1] if len(x) > 1 else np.nan)


    df = df.drop(columns=['types'])
    
    return df

def split_stats(df):
    """
    Splits the 'stats' column (which is a dictionary) into individual stat columns.
    
    The stats will be separated into the following columns: 'hp', 'attack', 'defense', 
    'special-attack', 'special-defense', and 'speed'.
    
    Args:
    - df (pd.DataFrame): The DataFrame containing the 'stats' column as a dictionary.
    
    Returns:
    - pd.DataFrame: The original DataFrame with the individual stat columns.
    """
    # Convert 'stats' dictionary into individual columns
    stats_df = df['stats'].apply(pd.Series)
    
    # Rename columns to match the specific stats
    stats_df.columns = ['hp', 'attack', 'defense', 'special-attack', 'special-defense', 'speed']
    
    # Concatenate the new stat columns with the original DataFrame
    df = pd.concat([df, stats_df], axis=1)

    
    # Drop the original 'stats' column
    df = df.drop(columns=['stats'])

    return df

def create_move_dummies(df):
    """
    Converts the 'moves' column into dummy columns, where each column represents a move, 
    with values True if the move exists for that Pokémon, and False otherwise.
    The new columns are named 'move_[MOVE_NAME]'.
    
    Args:
    - df (pd.DataFrame): The DataFrame containing the 'moves' column as a list of moves.
    
    Returns:
    - pd.DataFrame: The DataFrame with dummy columns for each move, and the original 'moves' column dropped.
    """
    # Get all unique moves across the entire DataFrame
    all_moves = set([move for sublist in df['moves'] for move in sublist])
    
    # Create a dummy column for each move
    for move in all_moves:
        df[f'move_{move}'] = df['moves'].apply(lambda moves: move in moves)
    
    # Drop the original 'moves' column
    df = df.drop(columns=['moves'])
    
    return df
    
df_all_pokemon = split_types(df_all_pokemon)
df_all_pokemon = split_stats(df_all_pokemon)
df_all_pokemon = create_move_dummies(df_all_pokemon)


df_all_pokemon

### Bronnenlijst

PokeAPI. (n.d.). PokeAPI v2 documentation. Van https://pokeapi.co/docs/v2